##RAG시작하기
랭체인은 여러 인공지능 애플리케이션 개발에 사용되지만 그중 가장 많이 사용되는 사용처를 언급하자
면 바로 RAG(Retrieval‐Augmented Generation) 일 것입니다. 직역하면 ‘검색 증강 생성’ 이라고 불리
는 이 기술은 사용자가 질문을 입력하면 입력한 질문으로 연관된 문서를 검색하고, 검색 결과를 바탕으로
답변하는 기술입니다.

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


###1. 텍스트 임베딩
인공지능 모델은 내부적으로 벡터 (vector) 연산으로 동작합니다. 여기서 벡터란 여러 개의 숫자가 특정
순서대로 나열된 것을 의미합니다. 예를 들어, 임의의 숫자 4 개가 나열된 벡터 [1, 0.2, 0.5, 7] 이 있다면 이
는 4 개의 원소를 가지는 벡터입니다. 이번 절에서 사용할 텍스트 인공지능 모델은 텍스트를 입력하면 주
어진 텍스트를 벡터로 반환합니다. 그리고 이 벡터를 이용하면 챗봇에 사용할 고성능의 검색 시스템을 빠
르고 쉽게 구현할 수 있습니다.
텍스트를 벡터화하는 과정 자체를 임베딩 (embedding) 이라고 부르고, 변환하고자 하는 단위에 따
라 텍스트를 벡터화하는 과정의 용어가 조금씩 다릅니다. 예를 들어 단어를 임베딩한다면 워드 임베딩
(word embedding) 이라 부르고, 어떤 문장이나 문서를 하나의 벡터로 변환한다면 문장 임베딩 또는 문서
임베딩이라고 부릅니다.

###2.코사인 유사도
벡터의 유사도를 구할 수 있는 가장 대표적인 방법으로 코사인 유사도 (cosine similarity) 가 있습니다. 코
사인 유사도는 두 벡터 간의 코사인 각도라는 개념을 이용해 두 벡터가 얼마나 유사한지를 나타내는 유사
도 값을 얻을 수 있는데, 값의 범위는 ‐1~1 사이로 유사도가 높을수록 1 에 가까운 값을 가집니다.

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

vec1 = np.array([0,1,1,1])
vec2 = np.array([1,0,1,1])
vec3 = np.array([2,0,2,2])

print('vec1과 vec2의 유사도: ', cos_sim(vec1, vec2))
print('vec1과 vec3의 유사도: ', cos_sim(vec1, vec3))
print('vec2와 vec3의 유사도: ', cos_sim(vec2, vec3))

vec1과 vec2의 유사도:  0.6666666666666667
vec1과 vec3의 유사도:  0.6666666666666667
vec2와 vec3의 유사도:  1.0000000000000002


파이썬 라이브러리인 Numpy 를 이용해 코사인 유사도를 계산하는 cos_sim 함수를 구현하고, 세 개의 임의의 벡터에 대해서 상호 유사도를 계산하여 출력합니다. 코사인 유사도는 벡터의 각 위치의 원소가 동일
하게 증가하는 경우에는 코사인 유사도 값이 1 이라는 특징이 있어 벡터 2 와 벡터 3 의 유사도는 최댓값인
1.0 이 나왔습니다. 지금까지 텍스트를 벡터로 만드는 임베딩이라는 기술이 있고, 코사인 유사도를 사용
하면 두 벡터의 유사도를 구할 수 있다는 내용을 살펴봤습니다. 그렇다면 챗봇을 만드는 데 벡터와 벡터
의 유사도 개념이 왜 필요할까요? 이는 앞으로 설명할 사용자의 질의로부터 가장 밀접하게 연관된 문서
를 찾는 검색 시스템을 구현하기 위해서입니다.

###3. 오픈AI임베딩과 코사인 유사도
이번 절에서는 챗봇에 사용할 검색 시스템의 구조를 이해하기 위해 OpenAI 의 Embedding API 와 코사인
유사도 개념을 이용하여 아주 간단한 검색 시스템을 구현해 보겠습니다. 필요한 라이브러리들을 임포트
하고 실습 환경에 OpenAI 키 값을 설정합니다.

In [ ]:
import os
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
os.environ['OPENAI_API_KEY'] = 'Openai_api_key'


랭체인의 OpenAIEmbeddings() 는 OpenAI 의 Embedding API 를 호출하는 역할을 합니다. 이를 이용하
면 주어진 텍스트로부터 OpenAI 의 모델을 이용하여 벡터를 만들어 줍니다. 랭체인이 내부적으로 사용하
고 있는 해당 API 에 대한 자세한 설명은 OpenAI 공식 문서에서 확인할 수 있습니다.
• OpenAI 공식 문서: https://platform.openai.com/docs/guides/embeddings/use‐cases
OpenAI 에서 제공하는 임베딩 모델은 여러 가지가 있지만 이 책에서는 그중 text-embedding-ada
-002 모델을 사용하여 ‘저는 배가 고파요’ 라는 문장을 임베딩하여 벡터로 변환하고, 변환한 벡터값을 출
력해 보겠습니다. OpenAIEmbeddings()로 임베딩 모델 객체인 embeddings를 선언하고 그 후에
는 embed_query(사용자의 입력)을 통해 사용자의 입력을 임베딩 벡터로 변환합니다.

In [ ]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
query_result = embeddings.embed_query("저는 배가 고파요")
print(query_result)

/tmp/ipython-input-4-3650863668.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")


[-0.016637360179694076, -0.02178889820399648, 0.015218060008672603, -0.027229550411783043, -0.036822973580909475, 0.011774940995488559, -0.034562605331125026, -0.006715396218388504, -0.023996698883951017, -0.016821344811453684, -0.008134697320732525, 0.010822169159143382, -0.010559335966606544, -0.022669389166164254, 0.011308411729489716, -0.004934699127060376, 0.012070628453507821, -0.0029240226430165585, 0.008134697320732525, -0.016137977579535355, 0.0013757691074475552, -0.014547834669211755, 0.01825378780625518, -0.01270142904691878, 0.00325256459934888, 0.006432850187165448, 0.005342090808323111, -0.019016004530273287, -0.009186031022202425, -0.0017347011662188633, 0.03451003776129511, -0.01644023644944463, 0.00011386029314690754, 0.003149073942476011, 0.007372479665391873, -0.005480078816481543, -0.006354000229404396, 0.0034726878055520963, -0.0037848023963125694, -0.0022505118304413413, -0.022432838827219824, -0.010362211305034551, 0.011354407887429618, -0.024732633685699258, 0.

In [ ]:
len(query_result)

1536

실행 결과로 다양한 실숫값이 나열된 벡터를 얻었습니다. text-embedding-ada-002 모델은 기본
적으로 텍스트를 임베딩하면 총 1,536 개의 숫자값이 나열된 벡터로 변환합니다. ‘저는 배가 고파요’ 라는
문장도 1,536 개의 숫자가 나열된 벡터값으로 변환됐습니다. 위 출력 결과에서는 지면의 한계로 벡터값
을 중략해서 표현했습니다. 해당 벡터값들이 어떤 의미인지는 사람이 해석하기는 어렵습니다. 여기서 확
인할 수 있는 것은 텍스트가 벡터로 변환됐다는 것과 벡터로 변환하고 나서 벡터 간 코사인 유사도를 구해
유사도가 높은지 테스트할 수 있다는 것입니다.
이제 유사도 테스트를 위해서 6 개의 데이터로 구성된 임의의 데이터프레임을 생성해 보겠습니다. 다음
코드는 6 개의 문장 데이터를 text 열에 할당하여 6 행 1 열로 구성된 데이터프레임인 df 를 만듭니다.

In [ ]:
data = ['저는 배가 고파요',
        '저기 배가 지나가네요',
        '굶어서 허기가 지네요',
        '허기 워기라는 게임이 있는데 즐거워',
        '스템에서 재밌는 거 해야지',
        '스팀에어프라이어로 연어구이 해먹을거야']

df = pd.DataFrame(data, columns = ['text'])

각 text 열에 존재하는 텍스트 데이터들을 get_embedding() 함수로 임베딩하여 벡터로 변환하고, 이를 새로운 embedding 열을 만들어 저장합니다.

In [ ]:
def get_embedding(text):
  return embeddings.embed_query(text)

df['embedding'] = df.apply(lambda row: get_embedding(row.text,), axis =1)
df

,text,embedding
0,저는 배가 고파요,"[-0.016637360179694076, -0.02178889820399648, ..."
1,저기 배가 지나가네요,"[-0.0032914344795325024, -0.027514765824463994..."
2,굶어서 허기가 지네요,"[-0.006181030746934311, -0.006950793503402665,..."
3,허기 워기라는 게임이 있는데 즐거워,"[-0.011354088532207011, -0.01170788027535161, ..."
4,스템에서 재밌는 거 해야지,"[-0.008349857740011572, -0.005234437225344748,..."
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.0021389091187001565, -0.030034279922246478..."


데이터프레임 df 에서 embedding 열의 값은 각 text 열에 있는 텍스트 데이터를 get_embedding() 함수로 얻은 벡터값입니다. 이제 임의의 입력이 들어오면 위 데이터프레임 df 에 존재하는 텍스트 데이터 중에서 가장 의미가 유사한 문장들을 반환하는 검색 시스템을 구현할 것입니다.
cos_sim() 함수는 앞서 살펴본 코사인 유사도를 계산하는 함수 cos_sim 을 다시 한번 구현한
것입니다. return_answer_candidate() 함수는 임의의 검색어가 들어오면 해당 검색어를
get_embedding() 함수로 임베딩하여 벡터로 변환하고, query_embedding 변수에 저장합니다. 그
다음 현재 데이터프레임 df 에 존재하는 모든 embedding 열의 벡터들과 코사인 유사도를 계산하여 코사인 유사도가 가장 높은 상위 3 개의 데이터를 찾아 반환합니다.

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_answer_candidate(df,query):
  query_embedding = get_embedding(query)
  df['similarity'] = df.embedding.apply(lambda x: cos_sim(np.array(x),
                                                          np.array(query_embedding)))

  top_three_doc = df.sort_values("similarity", ascending = False).head(3)
  return top_three_doc

return_answer_candidate() 함수를 사용하여 ‘아무것도 안 먹었더니 꼬르륵 소리가 나네’ 라는
문장과 임베딩 벡터값이 가장 유사한 상위 3 개의 데이터를 출력해 보겠습니다.

In [ ]:
sim_result = return_answer_candidate(df, '아무것도 안 먹었더니 꼬르륵 소리가 나네')
sim_result

,text,embedding,similarity
2,굶어서 허기가 지네요,"[-0.006181030746934311, -0.006950793503402665,...",0.836768
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.0021389091187001565, -0.030034279922246478...",0.815944
0,저는 배가 고파요,"[-0.016637360179694076, -0.02178889820399648, ...",0.812184


실행 결과를 보면 단어가 거의 겹치지 않는데도 배고픔이나 식사와 관련된 문장들이 있는 데이터가 출력된 것을 확인할 수 있습니다. 이처럼 텍스트 간의 의미적인 유사도를 계산하고자 한다면 Embedding API를 이용해 텍스트를 벡터로 변환하고, 코사인 유사도를 계산하는 것만으로도 꽤 좋은 성능의 검색 시스템을 구현할 수 있습니다. 이번 실습에서는 임베딩과 유사도라는 개념을 이해하기 위해 테이블 형태의 데이터를 다루는 파이썬 패키지인 pandas 를 이용하여 실습했지만, 현업에서는 pandas 대신에 벡터 데이터베이스라는 도구를 사용하는 경우가 많습니다.

### RAG 챗봇의 구조
RAG 챗봇은 기본적으로 참고할 문서들을 모두 임베딩하여 벡터로 변환한 후에 가지고 있습니다. 예를 들어 서울시 정책에 대해서 답변하는 RAG 챗봇을 구현한다고 가정해봅시다. 다양한 서울시 정책 문서를 각각 임베딩 벡터로 변환한 다음, 각 문서의 임베딩 벡터를 미리 가지고 있습니다. 여기까지가 챗봇이 실행되기 전에 미리 작업해야 하는 과정입니다.이후 챗봇을 가동하여 사용자의 질의가 입력으로 들어오면 사용자의 질의를 임베딩하여 벡터로 변환하고, 사용자의 질의 벡터와 이미 임베딩된 서울 정책의 임베딩 벡터들에 대해 각각 유사도를 계산합니다.
예를 들어 그림과 같이 ‘신혼부부를 위한 서울 정책이 궁금해’ 라는 문장이 들어오면 해당 문장을 벡터로 변환하고, 기존의 서울 정책 문서 벡터들과 각각 유사도를 계산하는 과정을 거칩니다. 여기서 벡터의 유사도가 높다는 것은 사용자의 질문인 ‘신혼부부를 위한 서울 정책’ 과 관련된 문서일 가능성이 높다는 것을 의미합니다. 수많은 서울 정책 문서에 대해서 유사도 점수를 모두 계산했다면 이 중에서 유사도 점수가 가장 높은 상위 3 개의 문서를 선택합니다.
그 후 적절한 답변을 작성할 수 있도록 3 개의 문서를 사용자의 질의 채팅과 함께 ChatGPT API 의 프롬프트로 전달합니다. 이제 답변을 잘 정리하는 것은 ChatGPT 의 몫입니다. ChatGPT 는 3 개의 문서를 검토하여 질의에 가장 적절한 답변을 사용자에게 반환하게 됩니다. 그럼 이어서 RAG 챗봇을 구현하기 위한 Langchain 의 도구들을 실습하고, 이후 실제로 RAG 챗봇을 구현해봅시다.